In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from env import get_connection
import os

In [4]:
# create helper function to get the necessary connection url.
def get_db_connection(database):
    return get_connection(database)

# connect to sql zillow database
url = "zillow"

# use this query to get data    
sql_query = '''SELECT *
            FROM predictions_2017
            JOIN properties_2017 on properties_2017.parcelid = predictions_2017.parcelid
            JOIN propertylandusetype on propertylandusetype.propertylandusetypeid = properties_2017.propertylandusetypeid;
            '''

# assign data to data frame
df = pd.read_sql(sql_query, get_connection(url))

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77580 entries, 0 to 77579
Data columns (total 65 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   id                            77580 non-null  int64  
 1   parcelid                      77580 non-null  int64  
 2   logerror                      77580 non-null  float64
 3   transactiondate               77580 non-null  object 
 4   id                            77580 non-null  int64  
 5   parcelid                      77580 non-null  int64  
 6   airconditioningtypeid         25007 non-null  float64
 7   architecturalstyletypeid      207 non-null    float64
 8   basementsqft                  50 non-null     float64
 9   bathroomcnt                   77580 non-null  float64
 10  bedroomcnt                    77580 non-null  float64
 11  buildingclasstypeid           15 non-null     float64
 12  buildingqualitytypeid         49810 non-null  float64
 13  c

In [16]:
df.isnull()

,id,parcelid,logerror,transactiondate,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,...,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock,propertylandusetypeid,propertylandusedesc
0,False,False,False,False,False,False,True,True,True,False,...,False,False,False,False,False,True,True,False,False,False
1,False,False,False,False,False,False,True,True,True,False,...,False,False,False,False,False,True,True,False,False,False
2,False,False,False,False,False,False,True,True,True,False,...,False,False,False,False,False,True,True,False,False,False
3,False,False,False,False,False,False,True,True,True,False,...,False,False,False,False,False,True,True,False,False,False
4,False,False,False,False,False,False,False,True,True,False,...,False,False,False,False,False,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77575,False,False,False,False,False,False,True,True,True,False,...,False,False,False,False,False,True,True,False,False,False
77576,False,False,False,False,False,False,True,True,True,False,...,False,False,False,False,False,True,True,False,False,False
77577,False,False,False,False,False,False,False,True,True,False,...,False,False,False,False,False,True,True,False,False,False
77578,False,False,False,False,False,False,True,True,True,False,...,False,False,False,False,False,True,True,False,False,False


In [11]:
# 1. Rename the columns to be more readable
df = df.rename(columns = {'bedroomcnt':'bedrooms', 'bathroomcnt':'bathrooms', 'calculatedfinishedsquarefeet':'area', 'taxvaluedollarcnt':'tax_value', 'yearbuilt':'year_built'})

In [13]:
# 2. Keep rows built in 2017
df.isna().sum()

bedrooms        11
bathrooms       11
area          8484
tax_value      493
year_built    9337
taxamount     4442
fips             0
dtype: int64

In [14]:
# Check for rows where the 'year_built' column is equal to 2017
rows_to_keep = df['year_built'] == 2017

# Create a new DataFrame containing only the rows where 'year_built' is 2017
df_filtered = df[rows_to_keep]

# Now df_filtered contains only the rows where 'year_built' is 2017
# You can use df_filtered for further analysis or save it to a new variable.


In [15]:
df.shape

(2152863, 7)